In [ ]:
## EfficientNet
## Google Colaboratory (https://colab.research.google.com) での学習・推論

## TF1.15 で学習・推論を実施します
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.15.3
!python -c 'import tensorflow as tf; print("Script says: {}".format(tf.__version__))'

In [ ]:
## 認証
!gcloud auth application-default login --no-browser
!gcloud auth login --no-browser

In [ ]:
## GCSからスクリプトをダウンロード
!(mkdir -p /content/effnet && gsutil -m rsync -r gs://path/to/efficientnet /content/effnet)

## ライブラリをインストール
!pip install cloud-tpu-profiler pycocotools

In [ ]:
## 設定
## -----------------------

## データセットのあるディレクトリ
%env DATASET=gs://path/to/dataset/

## ラベルのクラス数
%env NUM_CLASSES=4

## 学習済みモデルの保存先ディレクトリ
%env LOG_TO=gs://path/to/model/

## 入力画像サイズ（指定したサイズにリサイズされる）
%env INPUT_IMAGE_SIZE=1024

## リサイズ方式
%env RESIZE_METHOD=None

## バッチサイズ
%env BATCH_SIZE=32

## 学習画像の枚数
%env NUM_TRAIN_IMAGES=5288

## 評価画像の枚数
%env NUM_EVAL_IMAGES=740

## 指定したエポックごとにチェックポイントを書き出す
epoch_per_iter = 5

## 事前学習モデルのファイルパス
%env WARM_START_PATH=gs://path/to/noisy-student-efficientnet-b1/

## モデル名（efficientnet-b1, efficientnet-b7 など）
%env MODEL_NAME=efficientnet-b1

## 画像の水増し手法
%env AUG_NAME=None

## 画像水増し手法のサブポリシー
%env AUG_SUBPOLICY=zoom

## Mixup 水増しのアルファ値
%env MIXUP_ALPHA=0.0

## スケジューリング方式（下記のなかから1つ選択）
## --transfer_schedule: Head層→全レイヤ の学習エポック数 [head, all]
## --train_steps: 全レイヤ学習ステップ数
%env SCHEDULING=--transfer_schedule=5,10
#%env SCHEDULING=--train_steps=300000

## 学習率
%env BASE_LEARNING_RATE=0.016

%env OTHER_OPTIONS=--keep_checkpoint_max=0


## 自動設定項目
## -----------------------
import os, math
iter_per_loop = math.ceil(int(os.environ['NUM_TRAIN_IMAGES']) / int(os.environ['BATCH_SIZE']) * epoch_per_iter)
%env ITER_PER_LOOP=$iter_per_loop

## スケジュールから学習ステップ数を計算する
steps = int(os.environ['NUM_TRAIN_IMAGES']) * int(os.environ['SCHEDULING'].lstrip('-').split(',')[1]) / int(os.environ['BATCH_SIZE'])
print(f'Number of steps: {steps}')

In [ ]:
## EfficientNet Train
## ------------------------
!python /content/effnet/main_v4rc5.py \
--tpu=${TPU_NAME} \
--data_dir=${DATASET} \
--num_label_classes=${NUM_CLASSES} \
--model_dir=${LOG_TO} \
--model_name=${MODEL_NAME} \
--mode=train \
--input_image_size=${INPUT_IMAGE_SIZE} \
--train_batch_size=${BATCH_SIZE} \
${SCHEDULING} \
--num_train_images=${NUM_TRAIN_IMAGES} \
--iterations_per_loop=${ITER_PER_LOOP} \
--warm_start_path=${WARM_START_PATH} \
--resize_method=${RESIZE_METHOD} \
--augment_name=${AUG_NAME} \
--augment_subpolicy=${AUG_SUBPOLICY} \
--base_learning_rate=${BASE_LEARNING_RATE} \
--mixup_alpha=${MIXUP_ALPHA} \
${OTHER_OPTIONS}

In [ ]:
!pip install tensorflow==2.3
## EfficientNet Eval
## ------------------------
!python /content/effnet/main_v4rc5.py \
--tpu=${TPU_NAME} \
--data_dir=${DATASET} \
--num_label_classes=${NUM_CLASSES} \
--model_dir=${LOG_TO} \
--model_name=${MODEL_NAME} \
--mode=eval \
--input_image_size=${INPUT_IMAGE_SIZE} \
--eval_batch_size=8 \
--num_eval_images=${NUM_EVAL_IMAGES} \
--steps_per_eval=1000 \
--resize_method=${RESIZE_METHOD} \
--eval_iterator=latest \
${OTHER_OPTIONS}